In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
#import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
#import holidays
from datetime import datetime, timezone
from pyspark.ml.classification import RandomForestClassifier, BinaryLogisticRegressionSummary
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.pipeline import PipelineModel

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("US_Accidents") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR") #supress warnings
#spark.conf.set("spark.python.worker.reuse", "true")

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/25 14:35:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/25 14:35:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/25 14:35:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/25 14:35:09 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/11/25 14:35:09 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [3]:
#Read in Data
df = spark.read.parquet("updated_dataset.parquet")

In [4]:
# Use StringIndexer for encoding the 'Severity' column
indexer = StringIndexer(inputCol="Severity", outputCol="SeverityIndex")
df = indexer.fit(df).transform(df)

In [5]:
# Create list of features
feature_list = []
for col in df.columns:
    if col == 'Severity':
        continue
    elif col == 'SeverityIndex':
        continue
    elif col == 'Severity_Binary':
        continue
    else:
        feature_list.append(col)

In [6]:
# Split the data into train and test
splits = df.randomSplit([0.8, 0.2], 314)
train = splits[0]
test = splits[1]

In [7]:
from pyspark.sql import functions as F

# Step 1: Group by 'Severity' and count occurrences
class_counts = train.groupBy("Severity").count()

# Step 2: Use PySpark's max() function to find the maximum count
max_class_size = class_counts.agg(F.max('count')).collect()[0][0]

# Initialize a list to store the oversampled DataFrames for each class
oversampled_df_list = []

# Step 3: Oversample each class to match the maximum class size
for row in class_counts.collect():
    class_label = row['Severity']
    class_size = row['count']

    # Filter data for the current class
    class_data = train.filter(F.col("Severity") == class_label)

    if class_size < max_class_size:
        # Oversample the class to match the maximum class size
        class_data_oversampled = class_data.sample(withReplacement=True, fraction=max_class_size / class_size)
    else:
        # Retain the original data for the class if it's already at the maximum size
        class_data_oversampled = class_data

    # Add the oversampled data to the list
    oversampled_df_list.append(class_data_oversampled)

# Step 4: Combine all the oversampled DataFrames
oversampled_train = oversampled_df_list[0]  # start with the first one
for class_df in oversampled_df_list[1:]:
    oversampled_train = oversampled_train.union(class_df)

# Show the result of oversampling
oversampled_train.show()

# Step 5: Group by 'Severity' and count the occurrences in the oversampled DataFrame
oversampled_class_counts = oversampled_train.groupBy("Severity").count()

# Show the result
oversampled_class_counts.show()

+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|Severity|Temperature|Humidity|Pressure|Visibility|Wind_Speed|Precipitation|Weekday|Rush_Hour|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate_Indicator|Sex_ratio|Percent_Age_65_over|MedianIncome|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|Percent_Age_15-24|Severity_Binary|SeverityIndex|
+--------+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+-------------+
|       1|          9|      

+--------+-------+
|Severity|  count|
+--------+-------+
|       1|4523565|
|       3|4529901|
|       4|4525628|
|       2|4527457|
+--------+-------+



In [8]:
# Assemble data for logistic regression model
assembler = VectorAssembler(inputCols=feature_list,
                            outputCol="features")

oversampled_train = assembler.transform(oversampled_train)
test = assembler.transform(test)

In [9]:
# Standardize the predictors
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(oversampled_train)
scaledTrainData = scalerModel.transform(oversampled_train)
scaledTestData = scalerModel.transform(test)

In [10]:
# Fit logistic regression model with intercept
from pyspark.ml.classification import LogisticRegression

# instantiate the model
lr = LogisticRegression(labelCol='SeverityIndex',
                        featuresCol='scaledFeatures',
                        family="multinomial")

# Fit the model
lrModel = lr.fit(scaledTrainData)
print(f"Detected number of classes: {lrModel.numClasses}")

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

Detected number of classes: 4
Coefficients: DenseMatrix([[-1.45363390e-02, -7.58303790e-02,  2.91937057e-02,
              -3.74094275e-02, -1.24750825e-02,  6.96323113e-04,
              -3.32754450e-02, -4.13082899e-02, -1.03104336e-02,
               3.92842259e-02,  5.31494321e-01,  3.34042359e-01,
               2.01782267e-02, -1.65233702e-03, -1.00455642e-01,
               2.69371564e-02,  2.72164880e-02,  8.13547771e-03,
               3.88685492e-02, -1.11537199e-02, -2.82183257e-02,
               6.03423892e-02,  3.42269404e-03],
             [-1.47098705e-01, -7.93169750e-02,  2.34377584e-01,
               2.32671640e-03,  1.29782872e-01,  7.22770894e-03,
              -6.37089246e-02, -1.62654732e-02,  4.61424749e-02,
               3.38121905e-02,  2.21116128e-01,  1.55262087e-01,
              -1.20222472e-01, -1.21768660e-01,  4.69349218e-01,
               5.58349069e-02, -1.26020508e-01,  1.20803937e-03,
               2.88624360e-01,  5.25601320e-03, -2.63215556e-0

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = lrModel.transform(scaledTestData)
lrPred.select("prediction").distinct().show()

# Assuming `lrPred` is your predictions DataFrame
evaluator = MulticlassClassificationEvaluator(
    labelCol="Severity_Binary",
    predictionCol="prediction",
    metricName='truePositiveRateByLabel', 
    metricLabel=1.0
)

recall = evaluator.evaluate(lrPred)
print(f"Recall for label 1.0: {recall}")

accuracy = evaluator.evaluate(lrPred, {evaluator.metricName: "accuracy"})
print(f'Accuracy: {accuracy}')

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
|       3.0|
|       2.0|
+----------+



Accuracy: 0.17727210646270988


In [12]:
precision = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedPrecision'})
recall = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedRecall'})
f1_score = evaluator.evaluate(lrPred, {evaluator.metricName: 'f1'})

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

Precision: 0.6926750188675712
Recall: 0.17727210646270986
F1 Score: 0.2627434249030147


In [13]:
# Add a column to indicate correct or incorrect predictions
predictions = lrPred.withColumn(
    'is_correct', F.expr("CASE WHEN SeverityIndex = prediction THEN 1 ELSE 0 END")
)

# Calculate accuracy by class
accuracy_by_class = predictions.groupBy('SeverityIndex').agg(
    (F.sum('is_correct') / F.count('SeverityIndex')).alias('accuracy')
)

# Show per-class accuracy
accuracy_by_class.show()

+-------------+-------------------+
|SeverityIndex|           accuracy|
+-------------+-------------------+
|          0.0| 0.3596577196450649|
|          1.0| 0.5000532415212877|
|          3.0| 0.7769224847971673|
|          2.0|0.42437608173747976|
+-------------+-------------------+



In [14]:
distinct_labels = lrPred.select('SeverityIndex').distinct()
distinct_labels.show()


+-------------+
|SeverityIndex|
+-------------+
|          0.0|
|          1.0|
|          3.0|
|          2.0|
+-------------+



In [15]:
# Calculate metrics by class
labels = [row['SeverityIndex'] for row in lrPred.select('SeverityIndex').distinct().orderBy('SeverityIndex').collect()]

metrics = {}
for label in labels:
    # Filter predictions for the current label
    true_positive = lrPred.filter((F.col('SeverityIndex') == label) & (F.col('prediction') == label)).count()
    false_positive = lrPred.filter((F.col('SeverityIndex') != label) & (F.col('prediction') == label)).count()
    false_negative = lrPred.filter((F.col('SeverityIndex') == label) & (F.col('prediction') != label)).count()
    true_negative = lrPred.filter((F.col('SeverityIndex') != label) & (F.col('prediction') != label)).count()

    # Precision, Recall, and F1 Score
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    # Store metrics
    metrics[label] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

# Print metrics for each class
for label, metric in metrics.items():
    print(f"Class {label} - Precision: {metric['precision']:.4f}, Recall: {metric['recall']:.4f}, F1 Score: {metric['f1_score']:.4f}")

Class 0.0 - Precision: 0.8876, Recall: 0.3597, F1 Score: 0.5119
Class 1.0 - Precision: 0.3225, Recall: 0.5001, F1 Score: 0.3921
Class 2.0 - Precision: 0.0625, Recall: 0.4244, F1 Score: 0.1089
Class 3.0 - Precision: 0.0285, Recall: 0.7769, F1 Score: 0.0549
